## [209. Minimum Size Subarray Sum](https://leetcode.com/problems/minimum-size-subarray-sum/)

问题难度: &clubs; &clubs; &clubs;

### 问题描述

给定一个`n`个正整数的数组的数组和一个正整数`s`, 找到和大于`s`的**连续**子数组的最小长度, 如果没有的话, 直接返回`0`.

**示例**
```
Input: s = 7, nums = [2,3,1,2,4,3]
Output: 2
解释: 在问题的限制条件下, 子数组 [4,3] 有最小的数组长度
```

**进一步**:
如果找到`o(n)`的解决方案, 尝试一下找`O(nlogn)`时间复杂度的方案.

### 解题思路

使用一个二维数组维护从`i`到`j`的子数组之和, 遍历数组, 每次遍历数组需要更新第`i`列的值, 遇到`sum`大于等于`s`的, 可以判断其与当前最小长度的大小比较, 并基于此进行更新.

然后...超过内存限制了.😶其实在更新二维数组的过程中, 我们不难发现: 有很多数组是肯定没有用的, 那么我们可以考虑只存储一个一维数组用来记录即可, 因为其实之前的数组都已经没用了

然后...超过时间限制了.😶想想有木有`O(nlogn)`的方法, 考虑还是使用一个数组用来记录从`0`到`i`的和, 那么每次只需要使用二分进行查找`s+nums[i]`的值. 然后这样的话就不用每次都去更新`sum`的值了, 最后时间复杂度`O(nlogn)`即可.

参考了一下资料, 找到一种`O(n)`的方法, 使用两个指针, 分别记录当前子数组的前后索引, 然后`sum`存储当前子序列的和, 什么时候更新子序列呢? 当`sum < s`的时候, 右指针继续向右移动, `sum`增加; 当`sum >=s`时, 左指针向右移动, 根据情况进行更新最后的值.

### 代码

In [1]:
class Solution(object):
    def minSubArrayLen(self, s, nums):
        """
        :type s: int
        :type nums: List[int]
        :rtype: int
        """
        num_len = len(nums)
        min_num = 0
        sums = [[0 for _ in range(num_len)] for _ in range(num_len)]
        for i in range(num_len):
            num = nums[i]
            for j in range(i+1):
                if i >= 1:
                    sums[i][j] = sums[i-1][j] + num
                else:
                    sums[i][j] = num
                if sums[i][j] >= s and (min_num == 0 or i-j+1 <= min_num):
                    min_num = i - j + 1
        return min_num
    
    def minSubArrayLenI(self, s, nums):
        """
        :type s: int
        :type nums: List[int]
        :rtype: int
        """
        num_len = len(nums)
        min_num = 0
        sums = [0 for _ in range(num_len)]
        for i in range(num_len):
            num = nums[i]
            for j in range(i+1):
                if sums[j] < s:
                    sums[j] += num
                    if sums[j] >= s and (min_num == 0 or i-j+1 <= min_num):
                        min_num = i - j + 1
        return min_num
    
    def minSubArrayLenII(self, s, nums):
        """
        :type s: int
        :type nums: List[int]
        :rtype: int
        """
        def binarySearch(nums, s, left, right):
            #二分查找, 找到大于s的数字的最小下标,如果没有,返回0
            if right >= len(nums) or left >= len(nums):
                return -1
            if nums[left] >= s:
                return left
            mid = (left + right) // 2
            if nums[mid] > s:
                return binarySearch(nums, s, left, mid)
            elif nums[mid] < s:
                return binarySearch(nums, s, mid + 1, right)
            else:
                return mid
        min_num = 0
        num_len = len(nums)
        sums = [nums[i] for i in range(num_len)]
        for i in range(1, num_len):
            sums[i] = sums[i-1] + nums[i]
        # [2,3,1,2,4,3] 得到 [2,5,6,8,12,15], 接下来使用二分查找
        for i in range(num_len):
            num = nums[i]
            index = binarySearch(sums, s, i, num_len-1)
            new_index = index + 1 - i
            if index >= 0 and (min_num == 0 or new_index <= min_num):
                min_num = new_index
            s += num
        return min_num
    
    def minSubArrayLenIII(self, s, nums):
        """
        :type s: int
        :type nums: List[int]
        :rtype: int
        """
        now_sum = 0
        left = 0
        ans = 0
        for i in range(len(nums)):
            now_sum += nums[i]
            while now_sum >= s:
                now_sum -= nums[left]
                step = i - left + 1
                if ans == 0 or step <= ans:
                    ans = step
                left += 1
        return ans

In [3]:
sl = Solution()
s = 4
nums = [1,4,4]

print(sl.minSubArrayLen(s, nums))
print(sl.minSubArrayLenI(s, nums))
print(sl.minSubArrayLenII(s, nums))
print(sl.minSubArrayLenIII(s, nums))


1
1
1
1


## [210. Course Schedule II](https://leetcode.com/problems/course-schedule-ii/)

问题难度: &clubs; &clubs; &clubs;

### 问题描述


假设我们需要完成`n`门课, 标识为`0`到`n-1`.

一些课程有一些预备知识, 例如在上课程`1`之前需要先上课程`0`, 表示为`[0,1]`对.

给定课程的总数`n`和一系列的预备课程对, 返回如果希望能上完所有课程所需的顺序. 

可能有多个正确的顺序, 只需要返回其中一个即可. 如果无法完成, 直接返回一个空的数组.

**示例1**
```
Input: 2, [[1,0]] 
Output: [0,1]
解释: 一共需要上2门课. 在上课程1之前需要上课程0, 因此, 正确的顺序是:[0,1] .
```

**示例2**
```
Input: 4, [[1,0],[2,0],[3,1],[3,2]]
Output: [0,1,2,3] or [0,2,1,3]
解释: 一共需要上4门课程. 在上课程2之前, 需要上完课程1和课程2 , 而课程1和课程2需要在上完课程0之后才能上. 因此, 一个正确的课程顺序为:[0,1,2,3]. 另一个正确的顺序为: [0,2,1,3] .
```

**注意**:
- 给定的预备课程对是一个由一系列边构成的图, 而不是连接矩阵
- 可以假设在输入的预备课程中没有重复的边

### 解题思路

其实这一题是之前第207题的延伸和拓展, 我们
